In [5]:
import numpy as np 
import pandas as pd

from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from pandas import DataFrame 
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras import Input

import seaborn as sns
import cv2
import matplotlib.pyplot as plt
import time

In [6]:
train = pd.read_csv("D:/Nour/Semester 7/Machine Learning/Project/fashion-mnist_train.csv")
test = pd.read_csv("D:/Nour/Semester 7/Machine Learning/Project/fashion-mnist_test.csv")

In [7]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
train.isnull().sum().sum()

0

In [9]:
test.isnull().sum().sum()

0

In [50]:
X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]



In [51]:
X_test = test.iloc[:,1:]
y_test = test.iloc[:,0]


In [ ]:
from keras.callbacks import EarlyStopping

# Early stopping of patience 10
early_stopping = EarlyStopping(monitor='val_loss',patience=10,verbose = 1,restore_best_weights=True)
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255.0
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255.0

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# Creating a Sequential model, which is a linear stack of layers.
model = Sequential()

# Convolutional layer with 32 filters, each with a 3x3 kernel, using ReLU activation, and batch normalization.
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())

# Another convolutional layer with 32 filters, a 3x3 kernel, ReLU activation, and batch normalization.
# Additionally, apply dropout with a rate of 25% to reduce overfitting.
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

# Convolutional layer with 64 filters, a 3x3 kernel, ReLU activation, and batch normalization.
# Apply max pooling with a 2x2 pool size to down-sample the dimensions.
# Apply dropout with a rate of 25% to reduce overfitting.
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
    

# Convolutional layer with 128 filters, a 3x3 kernel, ReLU activation, and batch normalization.
# Apply dropout with a rate of 25% to reduce overfitting.
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

# Flatten the output to a 1D array for the fully connected layers.

model.add(Flatten())

# Fully connected layer with 512 neurons and ReLU activation, with batch normalization and drop out rate of 50%
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


# Another fully connected layer with 128 neurons and ReLU activation,with batch normalization and dropout with a rate of 50%.
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer with 10 neurons and softmax activation for multi-class classification.
model.add(Dense(10, activation='softmax'))


In [ ]:
# Compile the model using the Adam optimizer, sparse categorical crossentropy loss since the target is not encoded, and accuracy as the metric.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [23]:
# Train the model
epochs = 200

model.fit(X_train, y_train, batch_size=128, epochs=epochs, validation_data=(X_val, y_val), callbacks=[early_stopping])

accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/200
375/375 [==============================] - 225s 588ms/step - loss: 0.5372 - accuracy: 0.8137 - val_loss: 1.7591 - val_accuracy: 0.4357
Epoch 2/200
375/375 [==============================] - 214s 572ms/step - loss: 0.3377 - accuracy: 0.8791 - val_loss: 0.2827 - val_accuracy: 0.9018
Epoch 3/200
375/375 [==============================] - 214s 571ms/step - loss: 0.2800 - accuracy: 0.9009 - val_loss: 0.2474 - val_accuracy: 0.9104
Epoch 4/200
375/375 [==============================] - 200s 534ms/step - loss: 0.2544 - accuracy: 0.9108 - val_loss: 0.2559 - val_accuracy: 0.9070
Epoch 5/200
375/375 [==============================] - 194s 517ms/step - loss: 0.2278 - accuracy: 0.9189 - val_loss: 0.2491 - val_accuracy: 0.9119
Epoch 6/200
375/375 [==============================] - 188s 501ms/step - loss: 0.2033 - accuracy: 0.9260 - val_loss: 0.2094 - val_accuracy: 0.9260
Epoch 7/200
375/375 [==============================] - 220s 587ms/step - loss: 0.1907 - accuracy: 0.9320 - val_loss: 0

In [29]:
# Train model without early stopping
model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

accuracy1 = model.evaluate(X_test, y_test, verbose=0)[1]
print(f'Test Accuracy: {accuracy1 * 100:.2f}%')

Epoch 1/50
375/375 [==============================] - 236s 616ms/step - loss: 0.5444 - accuracy: 0.8124 - val_loss: 1.0143 - val_accuracy: 0.6623
Epoch 2/50
375/375 [==============================] - 231s 616ms/step - loss: 0.3356 - accuracy: 0.8810 - val_loss: 0.3068 - val_accuracy: 0.8917
Epoch 3/50
375/375 [==============================] - 264s 701ms/step - loss: 0.2777 - accuracy: 0.9004 - val_loss: 0.2448 - val_accuracy: 0.9130
Epoch 4/50
375/375 [==============================] - 244s 652ms/step - loss: 0.2420 - accuracy: 0.9133 - val_loss: 0.2239 - val_accuracy: 0.9170
Epoch 5/50
375/375 [==============================] - 269s 717ms/step - loss: 0.2262 - accuracy: 0.9194 - val_loss: 0.2021 - val_accuracy: 0.9263
Epoch 6/50
375/375 [==============================] - 266s 709ms/step - loss: 0.2034 - accuracy: 0.9278 - val_loss: 0.1954 - val_accuracy: 0.9275
Epoch 7/50
375/375 [==============================] - 282s 752ms/step - loss: 0.1874 - accuracy: 0.9315 - val_loss: 0.2273 -

The CNN model, with batch size = 128 and 50 epochs had the best performance of 94.36%

In [34]:
from keras.utils import to_categorical
X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255.0
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255.0

# y_train = to_categorical(y_train, 10)
# y_test = to_categorical(y_test, 10)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# Similar to previous model, but included more pooling layers
model2 = Sequential()
model2.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(BatchNormalization())

model2.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))

model2.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.25))
    
    
model2.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(512, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))
model2.add(Dense(128, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))
model2.add(Dense(10, activation='softmax'))

model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model2.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val))

accuracy3 = model2.evaluate(X_test, y_test, verbose=0)[1]
print(f'Test Accuracy: {accuracy3 * 100:.2f}%')

Epoch 1/50
375/375 [==============================] - 28s 65ms/step - loss: 0.9002 - accuracy: 0.6909 - val_loss: 1.7999 - val_accuracy: 0.3037
Epoch 2/50
375/375 [==============================] - 25s 68ms/step - loss: 0.5019 - accuracy: 0.8214 - val_loss: 0.3978 - val_accuracy: 0.8553
Epoch 3/50
375/375 [==============================] - 27s 72ms/step - loss: 0.4225 - accuracy: 0.8507 - val_loss: 0.3182 - val_accuracy: 0.8852
Epoch 4/50
375/375 [==============================] - 33s 89ms/step - loss: 0.3777 - accuracy: 0.8647 - val_loss: 0.2996 - val_accuracy: 0.8896
Epoch 5/50
375/375 [==============================] - 46s 122ms/step - loss: 0.3529 - accuracy: 0.8755 - val_loss: 0.2849 - val_accuracy: 0.8945
Epoch 6/50
375/375 [==============================] - 78s 208ms/step - loss: 0.3331 - accuracy: 0.8824 - val_loss: 0.3512 - val_accuracy: 0.8746
Epoch 7/50
375/375 [==============================] - 72s 192ms/step - loss: 0.3161 - accuracy: 0.8866 - val_loss: 0.3012 - val_accura

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255.0
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255.0

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)



In [55]:
from pandas import DataFrame
import math
X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32') / 255.0
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32') / 255.0

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)


# Similar to first model, but with different number of neurons
model1 = Sequential()
model1.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model1.add(BatchNormalization())

model1.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model1.add(BatchNormalization())
model1.add(Dropout(0.25))

model1.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))

model1.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model1.add(BatchNormalization())
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(256, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(128, activation='relu'))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(10, activation='softmax'))

model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model1.fit(X_train, y_train, batch_size=128, epochs=25, validation_data=(X_val, y_val))



In [35]:
start = time.time()

cnnaccuracy = model1.evaluate(X_test, y_test, verbose=0)[1]
score = model1.evaluate(X_test, y_test, steps=math.ceil(10000/32))
end = time.time()

print(f'Test Accuracy: {cnnaccuracy * 100:.2f}%, {score[1] * 100:.2f}%')

313/313 [==============================] - 21s 66ms/step - loss: 3.3780 - accuracy: 0.1000
Test Accuracy: 10.00%, 10.00%
